<a href="https://colab.research.google.com/github/elangbijak4/Riset-AI-Medis/blob/main/Training_Model_4_Deteksi_Foto_X_Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle pydicom tqdm imgaug

from google.colab import drive
drive.mount('/content/drive')

# Letakkan kaggle.json di ~/.kaggle
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c rsna-pneumonia-detection-challenge
!unzip -q stage_1_train_images.zip stage_1_train_labels.csv.zip

In [2]:
!pip install -q deeplake
import deeplake
ds = deeplake.load('hub://activeloop/nih-chest-xray-train')

In [3]:
import pandas as pd, pydicom, cv2
from imgaug import augmenters as iaa
import matplotlib.pyplot as plt

df = pd.read_csv('stage_1_train_labels.csv')
dfp = df[df['Target']==1]

def load_aug(fp):
    img = pydicom.dcmread(fp).pixel_array
    img = cv2.resize(img, (512,512))
    img = iaa.GaussianBlur((0,3.0))(image=img)
    return img

for _, r in dfp.sample(3).iterrows():
    img = load_aug(f"stage_1_train_images/{r.patientId}.dcm")
    x,y,w,h = [int(r[c]) for c in ['x','y','width','height']]
    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
    plt.imshow(img, cmap='gray'); plt.show()

In [ ]:
import os
import pydicom
import cv2

os.makedirs("rsna_png/train", exist_ok=True)
for pid in dfp.patientId.unique():
    d = pydicom.dcmread(f"stage_1_train_images/{pid}.dcm").pixel_array
    cv2.imwrite(f"rsna_png/train/{pid}.png", d)

In [ ]:
!git clone https://github.com/limingwu8/Pneumonia-Detection
%cd Pneumonia-Detection
pip install -q -r requirements.txt
python train.py --dataset ../rsna_png/train --annotations ../stage_1_train_labels.csv

1.   Download RSNA via Kaggle API
2.   Visualisasi pipeline: load DICOM, augment blur, gambarkan bounding box
1.   Konversi ke PNG untuk siapkan folder input
2.   Fine-tune Faster‑R‑CNN dengan bounding box detection, evaluasi mAP